In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys
import tensorflow_probability as tfp

In [2]:
def build_model(width, height, depth, classes):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => POOL layer set
		Conv2D(30, (3, 3), padding="same", input_shape=inputShape),
		Activation("relu"),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(50, (3, 3), padding="same"),
		Activation("relu"),
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(200),
		Activation("relu"),
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

# # define cnn model
# def build_model(w, h, d, c):
#   model = Sequential()
#   model.add(Conv2D(30, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
#   model.add(MaxPooling2D((2, 2)))
#   model.add(Conv2D(50, (3, 3), activation='relu'))
#   model.add(Flatten())
#   model.add(Dense(200, activation='relu', kernel_initializer='he_uniform'))
#   model.add(Dense(10, activation='softmax'))
#   return model

In [3]:
def step(X, y, model):
  # keep track of our gradients
  with tf.GradientTape() as tape:
		# make a prediction using the model and then calculate the
		# loss
    pred = model(X)
    loss = categorical_crossentropy(y, pred)
	# calculate the gradients using our tape and then update the
	# model weights
  grads = tape.gradient(loss, model.trainable_variables)
  return(grads)
  # print(type(grads))
  # print(grads)
  # opt.apply_gradients(zip(grads, model.trainable_variables))

In [4]:
# initialize the number of epochs to train for, batch size, and
# initial learning rate
EPOCHS = 1
BS = 32
INIT_LR = 0.01
# load the MNIST dataset
print("[INFO] loading MNIST dataset...")
((trainX, trainY), (testX, testY)) = mnist.load_data()
# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# # load dataset
# (trainX, trainY), (testX, testY) = mnist.load_data()
# # reshape dataset to have a single channel
# trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
# testX = testX.reshape((testX.shape[0], 28, 28, 1))
# #Scale/Normalize
# trainX = trainX.astype("float32") / 255.0
# testX = testX.astype("float32") / 255.0
# # one hot encode target values
# trainY = to_categorical(trainY)
# testY = to_categorical(testY)

[INFO] loading MNIST dataset...


In [5]:
num_machines = 10
num_train = int(trainX.shape[0])
num_train_per_machine = int(num_train/num_machines)

#np.random.shuffle(trainX)

#Split training and test data among 10 machines
for i in range(num_machines):
  globals()['X_train_%s' % str(i+1)] = trainX[i*num_train_per_machine: (i+1)*num_train_per_machine]
  globals()['y_train_%s' % str(i+1)] = trainY[i*num_train_per_machine: (i+1)*num_train_per_machine]


In [6]:
# build our model and initialize our optimizer
print("[INFO] creating model...")
model = build_model(28, 28, 1, 10)
opt = SGD(learning_rate=INIT_LR)

[INFO] creating model...


In [7]:
model.compile(optimizer=opt, loss=categorical_crossentropy,
	metrics=["acc"])
# now that the model is compiled we can compute the accuracy
(loss, acc) = model.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))

313/313 [==============================] - 4s 12ms/step - loss: 2.3106 - acc: 0.0714
[INFO] test accuracy: 0.0714


In [8]:
label_switch_attack_machines = [3]

loss_list = []
acc_list = []

for i in range(1000):
  print("------------------------"+str(i)+"-----------------------------")
  if(i%10 == 0):
    (loss, acc) = model.evaluate(testX, testY)
    print("[INFO] test accuracy: {:.4f}".format(acc))
    loss_list.append((i,loss))
    acc_list.append((i,acc))

  gradient_mean = []
  sum = [0] * 8
  flag = [0] * 8
  #For each machine, select a batch of 32 training data and perform one feedford and get gradients
  for machine in range(10):
    index = np.random.choice(globals()['X_train_%s' % str(machine+1)].shape[0], 32, replace=False)
    train_X_batch = globals()['X_train_%s' % str(machine+1)][index].copy()
    #train_X_batch = train_X_batch.reshape(32,28,28,1)
    train_y_batch = globals()['y_train_%s' % str(machine+1)][index].copy()
    # print(train_y_batch)
    if(machine in label_switch_attack_machines):
      print(machine)
      for batch_element in range(32):
        for pos in range(10):
          if(train_y_batch[batch_element][pos]==1):
            train_y_batch[batch_element][pos]=0.
            train_y_batch[batch_element][9-pos]=1.

    globals()['grads_mean_%s' % str(machine+1)] = step(train_X_batch, train_y_batch, model)

  #Getting gradient mean
  for machine in range(10):
    for tensor in range(len(globals()['grads_mean_%s' % str(machine+1)])):
      if(flag[tensor] == 0):
        #print(machine)
        sum[tensor] = globals()['grads_mean_%s' % str(machine+1)][tensor]
        flag[tensor] = 1
      else:
        #print(machine)
        #print(sum[tensor])
        sum[tensor] += globals()['grads_mean_%s' % str(machine+1)][tensor]
        #print(sum[tensor])

  for element in sum:
    gradient_mean.append(element/10)

  # Calculate median, trimmed mean for gradient array from all machines and apply to model!!!!!! - Try reshape and then back again
  print("------------------------"+str(i)+"-----------------------------")
  # opt.apply_gradients(zip(gradient_mean, model.trainable_variables))
  print(len(gradient_mean))
  opt.apply_gradients(zip(gradient_mean, model.trainable_variables))



# print("------------------Mean------------------")
# print(gradient_mean)
	# show timing information for the epoch

------------------------0-----------------------------
313/313 [==============================] - 4s 12ms/step - loss: 2.3106 - acc: 0.0714
[INFO] test accuracy: 0.0714
3
------------------------0-----------------------------
8
------------------------1-----------------------------
3
------------------------1-----------------------------
8
------------------------2-----------------------------
3
------------------------2-----------------------------
8
------------------------3-----------------------------
3
------------------------3-----------------------------
8
------------------------4-----------------------------
3
------------------------4-----------------------------
8
------------------------5-----------------------------
3
------------------------5-----------------------------
8
------------------------6-----------------------------
3
------------------------6-----------------------------
8
------------------------7-----------------------------
3
------------------------7------

In [9]:
(loss, acc) = model.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))
loss_list.append((i,loss))
acc_list.append((i,acc))

f = open("loss_mean_attack.txt", "w")
for element in loss_list:
    f.write(str(element) + "\n")
f.close()

f = open("acc_mean_attack.txt", "w")
for element in acc_list:
    f.write(str(element) + "\n")
f.close()

313/313 [==============================] - 4s 12ms/step - loss: 0.0881 - acc: 0.9856
[INFO] test accuracy: 0.9856
